# LLM - Finetune

Objetivo del notebook: realizar un finetune del modelo Mistral 7B usando LoRA sobre el corpus separado de Shakespeare.
Este notebook fue pensado para ser ejecutado desde Google Colab, de ahí viene la configuración del entorno.

## 0. Setup del entorno.

In [ ]:
!nvidia-smi

!pip install -q \
  "transformers>=4.45.0" \
  "datasets>=3.0.0" \
  "accelerate>=1.0.0" \
  "peft>=0.13.0" \
  "trl>=0.9.0" \
  bitsandbytes

## 1. Imports y configuración.

In [ ]:
import torch
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

## 2. Paths y carga del dataset.

In [ ]:
TEXT_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/redactor_train"
MODEL_DIR = "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-7b-instruct-v0.3"

paths = sorted(Path(TEXT_DIR).glob("*.txt"))
print("Archivos encontrados:", len(paths))

records = []
for p in paths:
    text = p.read_text(encoding="utf-8", errors="ignore").strip()
    if text:
        records.append({"text": text})

dataset = Dataset.from_list(records)
dataset

## 3. Tokenizer y modelo base.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.model_max_length = 1024

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("Modelo base cargado.")

## 4. Configuración LoRA.

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 5. Configuración del entrenamiento (TRL).

In [ ]:
train_config = SFTConfig(
    output_dir="/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-lora",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=10,
    save_strategy="epoch",
    bf16=torch.cuda.is_available(),
    packing=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=train_config,
    train_dataset=dataset,
    processing_class=tokenizer,
)

## 6. Entrenamiento y guardado.

In [ ]:
trainer.train()

# Guardar adapters LoRA
trainer.model.save_pretrained(
    "/content/drive/MyDrive/StoryWriter/Modelo_FineTuning/mistral-lora"
)